In [1]:
import pandas as pd
import time
import re
import datetime

In [2]:
import urllib.request
import urllib.parse
from bs4 import BeautifulSoup
# from konlpy.tag import Hannanum, Okt, Komoran, Kkma
from tqdm.notebook import tqdm

In [3]:
# 수집
def search(name_list, ds, de):
    baseUrl = 'https://search.naver.com/search.naver?&where=news&query='
    url = baseUrl + urllib.parse.quote_plus(name_list) 
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    subject = []
    date = []
    contents = []
    company = []
    
    page = 1
    
    while True:
        
        url = baseUrl + urllib.parse.quote_plus(name_list) + '&nso=so%3Ar%2Cp%3Afrom'+ str(ds) + 'to' + str(de) +'&start=' + str(page)
        html = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(html, 'html.parser')

        word = soup.find_all(class_='type01')[0]
        word = word.find_all('dl')

        title = soup.find_all(class_='_sp_each_title')

        com = soup.find_all(class_='_sp_each_source')

        date_re = re.compile(r'\d\d\d\d.\d\d.\d\d.')
        day = soup.find_all(class_='txt_inline')


        # 컨텐츠
        for words in word:
            contents.append(words.find_all('dd')[1].get_text())

        # 타이틀
        for tie in title:
            subject.append(tie.get_text())

        # 업체
        for coms in com:
            if coms.get_text()[-2:] == '선정':
                company.append(coms.get_text()[:-6])
            else:
                company.append(coms.get_text())

        # 시간
        for dates in day:
            try:
                match = date_re.search(dates.get_text())
                date.append(match.group())
            except:
                date_re2 = re.compile(r'\d')
                match = date_re2.search(dates.get_text())
                m = int(match.group())
                year = str(datetime.datetime.today().year)
                month = str(datetime.datetime.today().month)
                day = str(datetime.datetime.today().day-m)
                result = year + '.' + month + '.' + day + '.'
                date.append(result)
        
        print(page, end=' ')
        
        try:
            if soup.find_all(class_='next')[0].get_text() != '다음페이지':
                break
            else: 
                page += 10
        except:
            if soup.find(class_='next') != '다음페이지':
                break

    df = pd.DataFrame(list(zip(subject, date, contents, company)), columns = ['title', 'date','contents', 'company'])
    
    return df

In [3]:
# 전처리 2
def sentence(df):
    
    df['contents'] = df['contents'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]"," ")
    
    stopwords = ['을', '로', '의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다''의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

    okt = Okt()
    
    contents = []

    for sentence in tqdm(df['contents']):
        temp_x = []
        temp_x = okt.nouns(sentence)
        temp_x = [word for word in temp_x if not word in stopwords]
        contents.append(temp_x)
        
    return contents


In [5]:
# 단어 빈도수
def word_count(contents, name):
    wordlist = sum(contents, [])
    word_list = pd.Series(wordlist)
    result = word_list.value_counts()
    result.to_csv('data/기업 '+name+'.csv', mode='w')
    print(name + ' Save compelte')

In [6]:
result = pd.DataFrame(index=range(0,0), columns=['title', 'date','contents', 'company'])

In [7]:
com_list = pd.read_csv('data/com_list2.csv')
com_list['ds'] = [date - 10000 for date in com_list['date']]
com_list.rename(columns = {'date' : 'de'}, inplace = True)

In [8]:
com_list

,company,de,ds
0,VK,20060722,20050722
1,텔슨전자,20040810,20030810
2,이론테크,20030430,20020430
3,코리아링크,20030416,20020416
4,소프트윈,20021128,20011128


In [9]:
com_de = com_list['de'].tolist()
com_ds = com_list['ds'].tolist()
com_list = com_list['company'].tolist()
# com_list['company'] = [cmp_lst.strip() for cmp_lst in com_list]

In [1]:
for cmp_lst, ds, de in zip(com_list, com_ds, com_de):
    print(cmp_lst + ' Start' + ' ds' + str(ds) +' de' + str(de))
    try:
        df = search(cmp_lst, ds, de)
    except:
        continue 
        
    df_contents = sentence(df)
    word_count(df_contents, cmp_lst)
    result = pd.concat([result, df])
 

NameError: name 'com_list' is not defined

In [19]:
result.to_csv('data/네이버 기사.csv', mode='w', encoding='utf-8')

NameError: name 'result' is not defined

In [11]:
com_list

['VK', '텔슨전자', '이론테크', '코리아링크', '소프트윈']

In [3]:
df = pd.read_csv('./네이버 기사.csv')

In [4]:
df.head()

,Unnamed: 0,title,date,contents,company
0,0,"데코앤이, 장중 하락세, 전일대비 -14.74%... 이 시각 거래량 82만5062주",2019.11.19.,그래프 데코앤이 일봉 차트 표 거래원 상위 기준 이 시각 거래소에...,한국경제TV
1,1,"[장마감보고] 고가대비급락 종목 2위 데코앤이 섬유,의류,신발,호화품 업종내...",2019.11.19.,일 고가대비 급락종목 위 데코앤이 주가 리포트 일 시 분 기준 ...,공감신문
2,2,"데코앤이, 115원까지 올랐으나 115원 후퇴",2019.11.18.,일 데코앤이 하락종목 주가 리포트 일 시 분 현재 데코앤이 ...,내외경제tv
3,3,"[fnRASSI]장마감, 코스닥 하락 종목(데코앤이 -31.2% ↓)",2019.11.18.,일 코스닥 시장에서 하락한 종목으로 데코앤이 메지온...,파이낸셜뉴스
4,4,"데코앤이, 주가 하락 현재는 -24.64%... 이 시각 거래량 75만7821주",2019.11.18.,그래프 데코앤이 일봉 차트 표 거래원 상위 기준 이 시각 거래소에...,한국경제TV


In [5]:
df['contents'] = df['contents'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣]"," ")

In [6]:
df.head()

,Unnamed: 0,title,date,contents,company
0,0,"데코앤이, 장중 하락세, 전일대비 -14.74%... 이 시각 거래량 82만5062주",2019.11.19.,그래프 데코앤이 일봉 차트 표 거래원 상위 기준 이 시각 거래소에...,한국경제TV
1,1,"[장마감보고] 고가대비급락 종목 2위 데코앤이 섬유,의류,신발,호화품 업종내...",2019.11.19.,일 고가대비 급락종목 위 데코앤이 주가 리포트 일 시 분 기준 ...,공감신문
2,2,"데코앤이, 115원까지 올랐으나 115원 후퇴",2019.11.18.,일 데코앤이 하락종목 주가 리포트 일 시 분 현재 데코앤이 ...,내외경제tv
3,3,"[fnRASSI]장마감, 코스닥 하락 종목(데코앤이 -31.2% ↓)",2019.11.18.,일 코스닥 시장에서 하락한 종목으로 데코앤이 메지온...,파이낸셜뉴스
4,4,"데코앤이, 주가 하락 현재는 -24.64%... 이 시각 거래량 75만7821주",2019.11.18.,그래프 데코앤이 일봉 차트 표 거래원 상위 기준 이 시각 거래소에...,한국경제TV


In [7]:
stopwords = ['을', '로', '의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다''의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [8]:
contents = []

In [9]:
from eunjeon import Mecab

In [10]:
m = Mecab()

In [36]:
df['contents'].isna().sum()

0

In [35]:
df['contents'].dropna(axis=0, inplace=True)

In [37]:
for sentence in tqdm(df['contents']):
    temp_x = []
    temp_x = m.nouns(sentence)
    temp_x = [word for word in temp_x if not word in stopwords]
    contents.append(temp_x)

In [22]:
word_count(contents, 'VK')

VK Save compelte


In [38]:
contents

[['그래프',
  '데코',
  '앤',
  '일봉',
  '차트',
  '표',
  '거래원',
  '상위',
  '기준',
  '시각',
  '거래소',
  '데코',
  '앤',
  '이외',
  '남선',
  '알미늄',
  '뉴',
  '텍',
  '남선',
  '알',
  '미우',
  '티케이',
  '케미',
  '칼',
  '인트',
  '로메',
  '딕',
  '하락세'],
 ['일',
  '고가',
  '대비',
  '급락',
  '종목',
  '위',
  '데코',
  '앤',
  '주가',
  '리포트',
  '일',
  '시',
  '분',
  '기준',
  '데코',
  '앤',
  '장',
  '초반',
  '원',
  '상승',
  '원',
  '거래',
  '데코',
  '앤',
  '현재',
  '원',
  '일',
  '대비',
  '원',
  '하락'],
 ['일',
  '데코',
  '앤',
  '하락',
  '종목',
  '주가',
  '리포트',
  '일',
  '시',
  '분',
  '데코',
  '앤',
  '장',
  '초반',
  '원',
  '원',
  '거래',
  '거래일',
  '비교',
  '원',
  '하락',
  '가격',
  '데코',
  '앤',
  '이날'],
 ['일',
  '코스닥',
  '시장',
  '하락',
  '종목',
  '데코',
  '앤',
  '동일',
  '철강',
  '등',
  '금융',
  '전문',
  '기업',
  '씽크',
  '풀',
  '파이낸셜',
  '뉴스',
  '협업',
  '로봇',
  '기자'],
 ['그래프',
  '데코',
  '앤',
  '일봉',
  '차트',
  '표',
  '거래원',
  '상위',
  '기준',
  '시각',
  '거래소',
  '데코',
  '앤',
  '이외',
  '닉',
  '티코',
  '스메',
  '세원',
  '메지',
  '동일',
  '철강',
  '베',
  '스틸',
  '등',
  '하락세']

In [ ]:
wordlist = sum(contents, [])
word_list = pd.Series(wordlist)
result = word_list.value_counts()